# Машинное Обучение 2024, домашнее задание №1

Выполнил Дуников Константин [БИБ224]

## Часть 1. Подготовка


In [ ]:
# folium - библиотека для работы с географическими картами
%pip install folium
# numpy - быстрая математика на C с обёрткой на питон (вероятно уже установлена, но у меня чистый jupyter не подхватил)
%pip install numpy
# pandas - библиотека для работы с табличными данными (вероятно уже установлена, но у меня чистый jupyter не подхватил)
%pip install pandas
# seaborn - графики, диаграммы и т.п. (решил поэксперементировать и попробовать что-то, что не matplotlib)
# К концу выполнения понял, что так и не использовал; не придумал, куда вставить, folio или датафреймы тут приятнее смотрятся
# %pip install seaborn
# sklearn - библиотека, которая в принципе уже выполнила данную домашку
%pip install scikit-learn

In [ ]:
# Я за переносимость ноутбуков, несмотря на то, что код в них устаревает за 0.1с
# поэтому, дабы не парится с монтированием и загрузкой архива с датасетов
!wget https://storage.yandexcloud.net/hse-machine-learning/hw1_dataset.zip

--2024-10-10 20:40:22--  https://storage.yandexcloud.net/hse-machine-learning/hw1_dataset.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289279086 (276M) [application/zip]
Saving to: ‘hw1_dataset.zip’

hw1_dataset.zip     100%[===================>] 275.88M  14.1MB/s    in 22s     

2024-10-10 20:40:46 (12.4 MB/s) - ‘hw1_dataset.zip’ saved [289279086/289279086]



In [ ]:
!unzip hw1_dataset.zip # -> input_data.csv

Archive:  hw1_dataset.zip
  inflating: input_data.csv          


In [ ]:
import typing as tp # вот это нужно мне, я за статическую типизацию

import seaborn as sns
import pandas as pd
import numpy as np

import math

In [ ]:
# Мне показалось, что будет полезно ещё сделать пару констант
SPB_LAT_LON: tp.Tuple[float, float] = (59.9386, 30.3141)
MSK_LAT_LON: tp.Tuple[float, float] = (55.7522, 37.6156)

In [ ]:
# Функция, которая считывает csv в pandas DataFrame
# Не то, чтобы сильно нужная, но пусть будет
def load_dataframe(csv_path: str = 'input_data.csv') -> pd.DataFrame:
  return pd.read_csv(csv_path, sep=';')

In [ ]:
# Грузим данные из скачанного файла и немного на них смотрим
df = load_dataframe().loc[:100000] # У меня компуктер слабенький, для скорости я ограничил выборку

print(f'Dataframe has {df.shape[0]} rows and {df.shape[1]} columns')
df.head(n=5)

Dataframe has 100001 rows and 15 columns


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


In [ ]:
def display_flats(
    dataframe: pd.DataFrame = df,
    offset: int = 0, # сдвиг (первая квартира в датасете, которую надо отобразить)
    limit: int = 1000, # максимальное количество точек на карте
    start_location: tp.Tuple[float, float] = SPB_LAT_LON, # изначальная точка на карте
    start_zoom: int = 10, # изначальное значение зума на карте
) -> None:
  import folium as f
  from IPython.display import display

  map: f.Map = f.Map(location=start_location, zoom_start=start_zoom)

  for point in zip(dataframe['geo_lat'].loc[offset:limit+offset], dataframe['geo_lon'].loc[offset:limit+offset]):
    f.Marker(location=point).add_to(map)

  display(map)


In [ ]:
display_flats(dataframe=df)

## Часть 2. Предобработка данных

In [ ]:
# Где-то в процессе мы будем считаь расстояния между точками
# так как тут координаты, придётся немного попереводить величины

# Радиус Земли в КМ
EARTH_RADIUS_KM: int = 6371

# Переводим радианы в градусы
def rad_to_deg(rad: float) -> float:
  return rad * 180.0 / math.pi

# Переводим градусы в радианы
def deg_to_rad(deg: float) -> float:
  return deg * math.pi / 180.0

# Возвращает расстояние между точками в КМ
def haversine_distance(
    lat1: float, # Широта
    lon1: float, # Долгота
    lat2: float,
    lon2: float,
) -> float:
  delta_lat: float = lat2 - lat1
  delta_lon: float = lon2 - lon1

  medium_lat = (lat1 + lat2) * 0.5

  return EARTH_RADIUS_KM * math.sqrt(deg_to_rad(delta_lat) ** 2 + (math.cos(deg_to_rad(medium_lat)) * deg_to_rad(delta_lon)) ** 2)

def haversine_distance_tupled(
    locarion1: tp.Tuple[float, float],
    location2: tp.Tuple[float, float]
) -> float:
  return haversine_distance(locarion1[0], locarion1[1], location2[0], location2[1])

In [ ]:
# Если верить Яндексу - тут примерно 634
# Не самое точно, но примерно пойдёт
print(f"Расстояние от Москвы до Питера {haversine_distance_tupled(SPB_LAT_LON, MSK_LAT_LON)} км")

# P.S. Есть реализация haversine_distance внутри scikit, но я решил написать свою
# P.S.S. Реализована самая примитивная формула, на википедии есть ещё, но я решил не упарываться

Расстояние от Москвы до Питера 635.1377667439116 км


In [ ]:
# Найдём квартиры в СПб
df['nearby_spb'] = df.apply(lambda row: haversine_distance_tupled((row['geo_lat'], row['geo_lon']), SPB_LAT_LON) <= 20, axis=1)

In [ ]:
# Найдём квартиры в МСК
df['nearby_msk'] = df.apply(lambda row: haversine_distance_tupled((row['geo_lat'], row['geo_lon']), MSK_LAT_LON) <= 20, axis=1)

In [ ]:
# Посмотрим, насколько корректно отработал расчёт расстояния
display_flats(
    dataframe=df.loc[df['nearby_spb']],
    start_location=SPB_LAT_LON,
    start_zoom=7
)

Можно удостовериться, что квартиры в Москве не попали под условие `nearby_spb`

In [ ]:
def remove_useless_columns(dataframe: pd.DataFrame) -> pd.DataFrame:
  return dataframe.drop(
      columns=[
          'geo_lat',
          'geo_lon',
          'object_type',
          'postal_code',
          'street_id',
          'id_region',
          'house_id',
      ]
  )

In [ ]:
# Удаляем лишние колонки
df = remove_useless_columns(df)
# и смотрим, что там осталось
df.head(n=1)

,date,price,level,levels,rooms,area,kitchen_area,building_type,nearby_spb,nearby_msk
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False


Рассмотрим колонки и их типы

| Колонка | Тип |
| :------ | :-- |
| date    | числовой |
| price   | числовой |
| level   | числовой |
| levels  | числовой |
| rooms   | числовой, но так как есть `-1 = аппартаменты`, то категориальный |
| area    | числовой |
| kitchen_area | числовой |
| building_type | категориальный |
| nearby_spb | кажется, категориальный, но можно свести к числовому `False = 0`, `True = 1` |
| nearby_msk | кажется, категориальный, но можно свести к числовому `False = 0`, `True = 1` |

In [ ]:
# Разбивает категориальный признак в указанной колонке
# на несколько при помощи One Hot алгоритма
def one_hot_encode(
    dataframe: pd.DataFrame,
    column_to_encode: str
) -> pd.DataFrame:
  result: pd.DataFrame = dataframe.drop(columns=[column_to_encode])

  values = dataframe[column_to_encode].unique()
  for v in values:
    result[f'{column_to_encode}_{v}'] = dataframe[column_to_encode].apply(lambda x: 1.0 if x == v else 0.0)

  return result


In [ ]:
df = one_hot_encode(df, 'building_type')
df = one_hot_encode(df, 'nearby_msk')
df = one_hot_encode(df, 'nearby_spb')
df = one_hot_encode(df, 'rooms')

In [ ]:
df.head(n=1)

,date,price,level,levels,area,kitchen_area,building_type_0,building_type_3,building_type_2,building_type_4,...,nearby_spb_False,nearby_spb_True,rooms_1,rooms_3,rooms_2,rooms_-1,rooms_4,rooms_5,rooms_7,rooms_6
0,2021-01-01,2451300,15,31,30.3,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Про дополнительные числовые признаки

`количество дней со дня первого наблюдения (разница между датами объявлений)` - кажется тут речь о количестве дней между появлением и закрытием объявления о продаже, но я вижу условную дату подачи, но дату закрытия - нет
Вероятно, тут записи за каждый день, но я не могу сказать, что две даже одинаковые записи с разной датой относятся к одной квартире (нет flat_id, чтобы выполнить сопоставление, иначе мне кажется неочевидным [можно обсуждать])

In [ ]:
df = df.drop(columns=['date'])

`Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме` - с этим признаком я в целом согласен
Но считаю, что он не замени ни `level`, ни `levels`, поэтому просто добавлю его рядом

In [ ]:
df['level_to_levels'] = df.apply(lambda row: float(row['level']) / np.max([float(row['levels']), 1.0]), axis=1)

In [ ]:
# Теперь о нормировании
# функция, которая нормирует числовые признаки также, как StandartScaller
def scale_series(series: pd.Series) -> pd.Series:
  mean: float = np.mean(series)
  deviation: float = np.std(series)
  return series.apply(lambda x: (x - mean) / deviation)

In [ ]:
def normilize_digital_features(dataframe: pd.DataFrame) -> None:
  for feature in ['level', 'levels', 'area', 'kitchen_area']:
    dataframe[feature] = scale_series(dataframe[feature])

In [ ]:
normilize_digital_features(df)
df.head(n=3)

,price,level,levels,area,kitchen_area,building_type_0,building_type_3,building_type_2,building_type_4,building_type_5,...,nearby_spb_True,rooms_1,rooms_3,rooms_2,rooms_-1,rooms_4,rooms_5,rooms_7,rooms_6,level_to_levels
0,2451300,1.705238,2.709210,-0.945557,0.153269,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.483871
1,1450000,-0.229633,-0.863806,-0.836819,0.308726,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
2,10700000,-0.423120,0.235583,1.257407,0.464182,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.307692


In [ ]:
# Немного про подсчёт расстояния между объектами для knn

euclidian_distance: tp.Callable[[pd.Series, pd.Series], float] = lambda x, y: np.sum((y - x) ** 2) ** 0.5
manhattan_distance: tp.Callable[[pd.Series, pd.Series], float] = lambda x, y: np.sum(np.absolute(y - x))

In [ ]:
class KNNRegressor:
  def __init__(
      self,
      k: int = 5,
      distance_calcer: tp.Callable[[pd.Series, pd.Series], float] = euclidian_distance
  ) -> None:
    self.k = k
    self.distance_calcer = distance_calcer

    self.X = None
    self.Y = None

  def fit(
      self,
      X: pd.DataFrame,
      Y: pd.Series
  ) -> None:
    self.X = X
    self.Y = Y

  # Тут подсчёт только для одной строки, можно расширить, но мне так легче представлять
  # Возвращает предполагаемую цену)
  def predict(
      self,
      x: pd.Series
  ) -> float:
    distances: pd.Series = self.X.apply(lambda row: self.distance_calcer(row, x), axis=1)
    k_nearest_indexes = np.argsort(distances)[:self.k]
    return np.mean(self.Y.loc[k_nearest_indexes])

In [ ]:
# И прежде чем идти дальше к линейной регрессии, хочу потестить knn, поэтому начнём с mse
def mse(
    expected: pd.Series,
    real: pd.Series
  ) -> float:
  return np.sum((real - expected) ** 2) / real.size

In [ ]:
# И ещё mae
def mae(
    expected: pd.Series,
    real: pd.Series
) -> float:
  return np.sum(np.abs(real - expected)) / real.size

In [ ]:
# rmse....
def rmse(
    expected: pd.Series,
    real: pd.Series
  ) -> float:
  return mse(expected, real) ** 0.5

In [ ]:
X: pd.DataFrame = df.drop(columns='price')
Y: pd.Series = df['price']

# Учебная выборка - 75%
# Тестовая выборка - 25%

mid: int = df.shape[0]*3/4

X_train: pd.DataFrame = X.loc[:mid]
Y_train: pd.Series = Y.loc[:mid]

X_test: pd.DataFrame = X.loc[mid:]
Y_test: pd.Series = Y.loc[mid:]

In [ ]:
knn = KNNRegressor()
knn.fit(X_train, Y_train)

Y_real = X_test.apply(lambda row: knn.predict(row), axis=1)

print(f'MSE for my KNN is {mse(Y_test, Y_real)}')
print(f'MAE for my KNN is {mae(Y_test, Y_real)}')

MSE for my KNN is 9129840767610.32
MAE for my KNN is 1823632.212


На самом деле, MSE кажется немного неприятным (какое-то уж очень биг число)
Но если просто посмотреть разницу первых 10 элементов (ожидаемое - полученное), то ращница ещё хуже (я надеялся на получше)

In [ ]:
Y_test[:10] - Y_real[:10]

,0
751,957760.0
752,1412026.6
753,-248000.0
754,-186000.0
755,548158.6
756,5188602.0
757,1470000.0
758,-375756.8
759,-1081000.0
760,-630000.0


In [ ]:
# Возможно проблема в Евклидовом расстоянии между объектами
# Попробуем Манхеттенское
knn = KNNRegressor(k=5, distance_calcer=manhattan_distance)
knn.fit(X_train, Y_train)

Y_real = X_test.apply(lambda row: knn.predict(row), axis=1)

print(f'MSE for my KNN [with manhatten distance] is {mse(Y_test, Y_real)}')
print(f'MAE for my KNN [with manhatten distance] is {mae(Y_test, Y_real)}')

MSE for my KNN [with manhatten distance] is 8353829676661.165
MAE for my KNN [with manhatten distance] is 1808213.3872


In [ ]:
Y_test[:10] - Y_real[:10]

,0
751,957760.0
752,1412026.6
753,-464000.0
754,-186000.0
755,841820.0
756,4560106.0
757,1070000.0
758,-525756.8
759,-1081000.0
760,-630000.0


Кажется стало получше 😍

In [ ]:
# Чисто ради интереса - что нам предоставляет scikit
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)

Y_real = knn.predict(X_test)

print(f'MSE for my KNN [scikit impl] is {mse(Y_test, Y_real)}')
print(f'MAE for my KNN [scikit impl] is {mae(Y_test, Y_real)}')

MSE for my KNN [scikit impl] is 15254416295303.84
MAE for my KNN [scikit impl] is 2451381.64


Удивительно, но MSE от scikit в 10 раз больше, чем у моих решений
(да, тут микровыборка, но всё же)

In [ ]:
# Теперь поговорим о более серьёзных вещах - линейная регрессии

# Тут тоже хотел вынести метод оптимизации вовне (как было с методами рассчёта расстояния в knn)
# но я не силён в питоне и без интерфейсов мне тут грустно, поэтому всё в одно
# и даже без enum-ов(
class LinearRegression:
  def __init__(
    self,
    learning_rate: float = 0.01,
    optimization: str = 'sdg',
    epsilon: float = 10 ** -8,
    decay_rate: float = 0.9,
    max_iterations: int = 400,
  ) -> None:
    self.learning_rate = learning_rate
    self.epsilon = epsilon
    self.decay_rate = decay_rate
    self.max_iterations = max_iterations

    self.optimizer: tp.Callable[[pd.DataFrame, pd.Series], None] = None
    if optimization == 'sdg':
      self.optimizer = lambda x, y: self._sdg(x, y)
    elif optimization == 'momentum':
      self.optimizer = lambda x, y: self._momentum(x, y)
    else:
      self.optimizer = lambda x, y: self._adagrad(x, y)

    self.weights: np.ndarray = None
    self.bias: float = 0.0

  def fit(
      self,
      X: pd.DataFrame,
      Y: pd.Series,
  ) -> None:
    self.weights = np.zeros(X.shape[1])
    self.optimizer(X, Y)

  def predict(
      self,
      X: pd.DataFrame # Тут с Series не вышло(
  ) -> pd.Series:
    return np.dot(X, self.weights) + self.bias

  def _sdg(
      self,
      X: pd.DataFrame,
      Y: pd.Series
  ) -> None:
    for i in range(self.max_iterations):
      y_pred = self.predict(X)
      #error = mse(y_pred, Y) # Использую MSE как по заданию (он понимал, что делает)
      error = y_pred - Y

      self.weights -= self.learning_rate * np.dot(X.T, error) / X.shape[0]
      self.bias -= self.learning_rate * np.sum(error) / X.shape[0]

  def _momentum(
      self,
      X: pd.DataFrame,
      Y: pd.Series,
  ) -> None:
    v_w = np.zeros(X.shape[1])
    v_b = 0
    for i in range(self.max_iterations):
      y_pred = self.predict(X)
      error = y_pred - Y

      v_w = self.decay_rate * v_w
      v_w -= self.learning_rate * np.dot(X.T, error) / X.shape[0]

      v_b = self.decay_rate * v_b
      v_b -= self.learning_rate * np.sum(error) / X.shape[0]

      self.weights += v_w
      self.bias += v_b

  def _adagrad(
      self,
      X: pd.DataFrame,
      Y: pd.Series
  ) -> None:
    cache_w = np.zeros(X.shape[1])
    cache_b = 0

    for i in range(self.max_iterations):
      y_pred = self.predict(X)
      error = y_pred - Y

      cache_w += np.square(np.dot(X.T, error) / X.shape[0])
      cache_b += np.square(np.sum(error) / X.shape[0])
      self.weights -= self.learning_rate * np.dot(X.T, error) / X.shape[0] / ((cache_w + self.epsilon) ** 0.5)
      self.bias -= self.learning_rate * np.sum(error) / X.shape[0] / ((cache_b + self.epsilon) ** 0.5)

In [ ]:
# Ну а теперь пройдёмся по всем методам оптимизации и посмотрим, что тут вышло
for o in ['sdg', 'momentum', 'adagrad']:
  linreg = LinearRegression(optimization=o)
  linreg.fit(X_train, Y_train)

  Y_real = linreg.predict(X_test)

  print(f'MSE for my Linear Regression is {mse(Y_test, Y_real)} [{o} optimization]')
  print(f'MAE for my Linear Regression is {mae(Y_test, Y_real)} [{o} optimization]')
  print(Y_test[:5] - Y_real[:5])

MSE for my Linear Regression is 15270251772897.152 [sdg optimization]
MAE for my Linear Regression is 2821472.456414359 [sdg optimization]
751    2.204698e+05
752    3.426589e+06
753    1.391776e+06
754    7.554838e+05
755    3.073296e+06
Name: price, dtype: float64
MSE for my Linear Regression is 13437421847485.912 [momentum optimization]
MAE for my Linear Regression is 2418232.560403003 [momentum optimization]
751    8.132733e+05
752    3.592984e+06
753    3.231464e+06
754    1.529692e+06
755    1.542423e+06
Name: price, dtype: float64
MSE for my Linear Regression is 36469790807446.78 [adagrad optimization]
MAE for my Linear Regression is 4579925.098173886 [adagrad optimization]
751    5.549998e+06
752    7.878631e+06
753    2.899998e+06
754    1.799998e+06
755    2.849999e+06
Name: price, dtype: float64


Видно, что хуже всех тут работает adagard

knn как будто отработал лучше, но мне кажется, дело в размере выборки

In [ ]:
# Ну и посмотрим, как себя покажет scikit
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, Y_train)

Y_real = linreg.predict(X_test)

print(f'MSE for SciKit Linear Regression is {mse(Y_test, Y_real)}')
print(f'MAE for SciKit Linear Regression is {mae(Y_test, Y_real)}')

MSE for SciKit Linear Regression is 9549945803719.775
MAE for SciKit Linear Regression is 2049911.4158647708


Вот тут явно всё работает получше

А теперь я пойду и везде помимо подсчёта mse ещё попытаюсь успеть вставить хотя бы mae